In [59]:
#! pip install -r requirements.txt --quiet

# Creating and Deploying an Azure AI Agent Using SDK and MCP Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use the Model Context Protocol tool](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)

In [60]:
from os import environ
import time
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder, 
    McpTool,
    RequiredMcpToolCall,
    RunStepActivityDetails,
    SubmitToolApprovalAction,
    ToolApproval,
)

load_dotenv(override=True)

True

## Get MCP server configuration from environment variables

In [61]:
mcp_server_url = environ["MCP_SERVER_URL"]
mcp_server_label = environ["MCP_SERVER_LABEL"] 

## Create a project client

In [62]:
# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Initialize agent MCP tool

In [63]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
   # allowed_tools=['get_customers','get_orders','get_products']
)

print(f"Allowed tools: {mcp_tool.allowed_tools}")

Allowed tools: []


## Create an agent and a thread

In [64]:
instructions = """
You are a Transport Operations Agent that answers questions about trucking
loads, dispatch activity, and operational exceptions.

Your role is to interpret user questions, determine which MCP tool or tools
are required, and use those tools to retrieve factual operational data.
Base all responses strictly on the results returned by the MCP tools.

Objectives:
- Select and call the appropriate MCP tool(s) to retrieve operational facts.
- Use MCP tool results as the sole source of truth.
- Do NOT infer, invent, or assume operational details that are not returned.
- If required information is missing, clearly state that it is not available.
- Return concise, structured responses suitable for operational review.

Scope:
- Trucking loads and shipment identifiers
- Dispatch assignments and related metadata
- Operational exceptions (e.g., delays, breakdowns, violations)
- Event timelines derived from returned records


Behavioral guidance:
- If a question refers to a specific load, retrieve its operational context
  before answering.
- If a question focuses on issues or delays, retrieve exception records.
- If a request is ambiguous, ask a clarifying question before calling tools.
- Do not expose database structure, schemas, or internal identifiers beyond
  what is returned by the tools.

Response style:
- Clear and factual
- Operationally focused
- No speculation or narrative embellishment
"""




agent = project_client.agents.create_agent(
    model=environ["MODEL_DEPLOYMENT_NAME"],
    name="TansportOperationsAgent",
    instructions=instructions,
    tools=mcp_tool.definitions
)

print(f"Created agent, ID: {agent.id}")



thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created agent, ID: asst_x52TCmXk1g9RDhCHXjUuVn8M
Created thread, ID: thread_scLbEM2fl8kPwLsubpQl0ZdT


In [65]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="List all loads that have recorded operational exceptions related to Breakdown. Include the load ID and a brief description of the exception if available.",
)
print(f"Created message, ID: {message.id}")



Created message, ID: msg_OUzVe4T30QY4dh43ntw4dynw


In [66]:
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            project_client.agents.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Display run steps and tool calls
run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)

Created run, ID: run_B2UpEHw7I2W6Z6qqp5xMw34i
Approving tool call: {'id': 'call_sPidgGU1u1szmFbz5hyywMD9', 'type': 'mcp', 'arguments': '{"exception_type":"Breakdown"}', 'name': 'get_exceptions_by_type', 'server_label': 'TransportOperations'}
tool_approvals: [{'tool_call_id': 'call_sPidgGU1u1szmFbz5hyywMD9', 'approve': True, 'headers': {}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


In [67]:
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
print("\nConversation:")
print("-" * 50)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role.upper()}: {last_text.text.value}")
        print("-" * 50)


Conversation:
--------------------------------------------------
USER: List all loads that have recorded operational exceptions related to Breakdown. Include the load ID and a brief description of the exception if available.
--------------------------------------------------
ASSISTANT: I encountered an error while attempting to retrieve the requested data. I cannot currently provide the list of loads with "Breakdown" exceptions. Please try again later or specify a different request.
--------------------------------------------------


In [68]:
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
